## Context

Context 跨 API 边界和 goroutine 携带截止日期、取消信号和其他请求范围的值

https://gobyexample.com/context

```go
package main

import (
    "fmt"
    "net/http"
    "time"
)

func hello(w http.ResponseWriter, req *http.Request) {

    ctx := req.Context()
    fmt.Println("server: hello handler started")
    defer fmt.Println("server: hello handler ended")

    select {
    case <-time.After(10 * time.Second):
        fmt.Fprintf(w, "hello\n")
    case <-ctx.Done():

        err := ctx.Err()
        fmt.Println("server:", err)
        internalError := http.StatusInternalServerError
        http.Error(w, err.Error(), internalError)
    }
}

func main() {

    http.HandleFunc("/hello", hello)
    http.ListenAndServe(":8090", nil)
}
//----------------------------------------------------------------
```
```output
$ go run context-in-http-servers.go &


$ curl localhost:8090/hello
server: hello handler started
^C
server: context canceled
server: hello handler ended
```